In [1]:
from os import path
from enmspring.graphs import GraphAgent
from enmspring import PDB
from enmspring import atom
import MDAnalysis as mda
rootfolder = '/home/yizaochen/codes/dna_rna/fluctmatch_sequence'
enmroot = '/home/yizaochen/codes/dna_rna/enmspring'

### Part 1: Initialize

In [106]:
host = 'a_tract_21mer'
g_agent = GraphAgent(host, rootfolder)

/home/yizaochen/codes/dna_rna/fluctmatch_sequence/a_tract_21mer/bdna+bdna/pd_dfs exists


### Part 2: Show crd in VMD

In [107]:
g_agent.vmd_show_crd()

vmd -cor /home/yizaochen/codes/dna_rna/fluctmatch_sequence/a_tract_21mer/bdna+bdna/input/bdna+bdna.nohydrogen.crd


### Part 3: Eigen-decomposition

In [108]:
g_agent.build_node_list_base()
print(f"Thare are {g_agent.n_node} nodes.")
g_agent.build_adjacency_from_df_st()
g_agent.build_degree_from_adjacency()
g_agent.build_laplacian_by_adjacency_degree()
g_agent.eigen_decompose()

Thare are 399 nodes.
Finish the setup for Laplaican matrix.


### Part 4: Select Eigenvector

In [109]:
sele_id = 3
scale_factor = 10.
eigv = g_agent.get_eigenvector_by_id(sele_id)
eigv_scale = scale_factor * eigv

In [110]:
eigv_scale.max()

2.9610350647006465

In [111]:
eigv_scale.min()

-4.001294213177385

### Part 5: Convert crd to pdb

In [112]:
u = mda.Universe(g_agent.npt4_crd, g_agent.npt4_crd)
npt4_pdb = path.join(g_agent.input_folder, 'bdna+bdna.nohydrogen.pdb')
with mda.Writer(npt4_pdb, bonds=None, n_atoms=u.atoms.n_atoms) as pdbwriter:
    pdbwriter.write(u.atoms)
print(f'vim {npt4_pdb}')

vim /home/yizaochen/codes/dna_rna/fluctmatch_sequence/a_tract_21mer/bdna+bdna/input/bdna+bdna.nohydrogen.pdb


/home/yizaochen/miniconda3/envs/enmspring/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:722: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/yizaochen/miniconda3/envs/enmspring/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1026: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/yizaochen/miniconda3/envs/enmspring/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1026: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/yizaochen/miniconda3/envs/enmspring/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1026: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  warnings.warn("Found no information for attr: '{}'"


### Part 6: Read in to pdbreader

In [113]:
reader = PDB.PDBReader(npt4_pdb, skip_header=9, skip_footer=2, withfragid=True)

In [75]:
#for atg in reader.atomgroup:
#    print(atg.tempFactor)

### Part 7: Add two dummy atoms to keep color scale in [-1,1]

In [114]:
minimum = eigv_scale.min()
maximum = -eigv_scale.min()

In [101]:
minimum = -eigv_scale.max()
maximum = eigv_scale.max()

In [115]:
# get serial and resid of the last atom
serial = reader.atomgroup[-1].serial + 1
resid = reader.atomgroup[-1].resid + 1
dummy1_data = ['ATOM', serial, 'S1', 'DUM', resid, 0.0, 0.0, 0.0, 0.0, minimum]
dummy2_data = ['ATOM', serial+1, 'S2', 'DUM', resid+1, 0.0, 0.0, 0.0, 0.0, maximum]
reader.atomgroup.append(atom.Atom(dummy1_data, False))
reader.atomgroup.append(atom.Atom(dummy2_data, False))

### Part 8: Get nodes idx map to pdb

In [116]:
for cgname, eigv_value in zip(g_agent.node_list, eigv_scale):
    atomid = g_agent.atomid_map[cgname]
    reader.atomgroup[atomid-1].set_tempFactor(eigv_value)

### Part 9: Output PDB for eigenvector

In [117]:
f_out = path.join(g_agent.input_folder, f'eigv_{sele_id}.pdb')
writer = PDB.PDBWriter(f_out, reader.atomgroup)
writer.write_pdb()
print(f'vim {f_out}')

Write PDB: /home/yizaochen/codes/dna_rna/fluctmatch_sequence/a_tract_21mer/bdna+bdna/input/eigv_3.pdb
vim /home/yizaochen/codes/dna_rna/fluctmatch_sequence/a_tract_21mer/bdna+bdna/input/eigv_3.pdb


### Part 10: Show PDB in vmd

In [118]:
g_agent.vmd_show_crd()
print(f'mol new {f_out} type pdb')

vmd -cor /home/yizaochen/codes/dna_rna/fluctmatch_sequence/a_tract_21mer/bdna+bdna/input/bdna+bdna.nohydrogen.crd
mol new /home/yizaochen/codes/dna_rna/fluctmatch_sequence/a_tract_21mer/bdna+bdna/input/eigv_3.pdb type pdb
